# Data Loading & Processing

In this notebook I will be accessing the [Oracle Autonomous Database](https://www.oracle.com/autonomous-database/) and gathering data from the Calls and Census Tables. I will then be processing the data in the notebook, resulting in one final dataframe which we can use to perform anything that we may wish to do.

In [1]:
##Logan Lauton
##Data Loading & Processing

##importing required packages 
import cx_Oracle
import datetime
import oracledb
import os
import pandas as pd
import requests

##connecting to the db as we did in grab_data.ipynb
os.environ.get('TNS_ADMIN')
connection = oracledb.connect(
    user="admin",
    password="aybM6CFL9gykHEKn",
    dsn = "ltl3y0m4d7of29l1_high",
    config_dir="./config",
    wallet_location="./config",
    wallet_password="phbUHx6CMjxQj75A")

cursor = connection.cursor()

##selecting everything from the calls & census tables and creating a df
rs = cursor.execute("SELECT * FROM CALLS")
df1 = pd.DataFrame(rs.fetchall())
rs = cursor.execute("SELECT * FROM CENSUS")
df2 = pd.DataFrame(rs.fetchall())

##making changes to Calls df by first naming columns appropriately
##before sorting the values via most recent date and changing 
##column types according to what is needed
df1.columns = ['Call Number' , 'Date/Time' , 'Address' , 'District' , 'Nature of Call' , 'Status' , 'Latitude' , 'Longitude', "Tract"]
df1[["Date/Time"]] = df1[["Date/Time"]].apply(pd.to_datetime)
df1 = df1.sort_values(by='Date/Time', ascending=False)
df1[["Tract"]] = df1[["Tract"]].apply(pd.to_numeric)

##making changes to Census df by first naming columns appropriately
##before adding in and altering columns with issues 
df2.columns = ['GEOID' , 'Tract' , 'Geometry' , 'Estimated Population' , 'Male Population' , 'Female Population' , 'Median Age' , 'Median Income', 'White Population', 'Black Population', 'Native Population', 'Hispanic Population', 'Income Poverty Level']
df2[["Tract","Estimated Population", "Male Population" ,"Female Population", "Median Age", "Median Income","White Population", "Black Population", "Native Population", "Hispanic Population","Income Poverty Level"]] = df2[["Tract","Estimated Population", "Male Population" ,"Female Population", "Median Age", "Median Income","White Population", "Black Population", "Native Population", "Hispanic Population","Income Poverty Level"]].apply(pd.to_numeric)
df2["Percent White"] = round((df2["White Population"] / df2["Estimated Population"])*100,3)
df2["Percent Black"] = round((df2["Black Population"] / df2["Estimated Population"])*100,3)
df2["Percent Native"] = round((df2["Native Population"] / df2["Estimated Population"])*100,3)
df2["Percent Hispanic"] = round((df2["Hispanic Population"] / df2["Estimated Population"])*100,3)
df2["Predominant  Tract Population"] = df2[['Percent White', 'Percent Black', 'Percent Native','Percent Hispanic']] \
    .idxmax(axis=1).str.replace('Percent ', '').tolist()

##making merged df that gives all of the census data for each call
df=df1.merge(df2,how="left",on="Tract")

##deleting individual dfs to gather back memory
del df1, df2

##storing specific cases where merge didn't properly assign census data
##then dropping empty cases accordingly
empty = df[df['GEOID'].isnull()]
df = df.dropna(subset=['GEOID'])


### UPDATING CENSUS TABLE

With this code the Table is now fully updated to include the correct data values for the hispanic population

In [29]:
import cx_Oracle
import datetime
import oracledb
import os
import pandas as pd
import requests

df3 = pd.read_csv("MilwaukeeCensus.csv")
df3 = df3[['GEOID', 'B03001_003E']]
df3.columns = ['GEOID','Hispanic Population']


os.environ.get('TNS_ADMIN')
connection = oracledb.connect(
    user="admin",
    password="aybM6CFL9gykHEKn",
    dsn = "ltl3y0m4d7of29l1_high",
    config_dir="./config",
    wallet_location="./config",
    wallet_password="phbUHx6CMjxQj75A")

cursor = connection.cursor()

# Define a function to update the 'CENSUS' table with values from the DataFrame
def update_census(row):
    query = "UPDATE CENSUS SET Hispanic_Pop = :Hispanic_Pop WHERE GEOID = :GEOID"
    cursor.execute(query, {'Hispanic_Pop': row['Hispanic Population'], 'GEOID': row['GEOID']})

# Loop through each row in the DataFrame and call the function to update the 'CENSUS' table
for index, row in df3.iterrows():
    update_census(row)

# Commit the changes
connection.commit()
